In [61]:
from typing import Any, List, Dict, Mapping, Optional
import json

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.requests import TextRequestsWrapper
from langchain.llms.base import LLM
import os
from volcengine.maas import MaasService, MaasException, ChatRole

# llm = CustomLLM()
# print(llm("who are you?"))
maas = MaasService('maas-api.ml-platform-cn-beijing.volces.com', 'cn-beijing')
maas.set_ak("AKLTNjU0MmRhMjRlNTViNGM1OGIwMzE2ZmIzYzQyNWM1ODk")
maas.set_sk("T0RjMVpUUTVaRE0xTlRBME5EQXlOMkUzWkdObU56UTFOekkyTlRNd1pqVQ==")

class CustomLLM(LLM):

    logging: bool = False
    output_keys: List[str] = ["output"]

    llm_type: str = "chatglm"

    @property
    def _llm_type(self) -> str:
        return self.llm_type

    def log(self, log_str):
        if self.logging:
            print(log_str)
        else:
            return
    
    def test_chat(self, maas, req):
        try:
            resp = maas.chat(req)
            print(resp)
            print(resp.choice.message.content)
            return resp.choice.message.content

        except MaasException as e:
            print(e)

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        self.log('----------' + self._llm_type + '----------> llm._call()')
        self.log(prompt)
        requests = TextRequestsWrapper()

        req = {
            "model": {
                "name": "chatglm2-pro",
            },
            "parameters": {
                "max_new_tokens": 1024,  # 输出文本的最大tokens限制
                "temperature": 0.8,  # 用于控制生成文本的随机性和创造性，Temperature值越大随机性越大，取值范围0~1
                "top_p": 0.8,  # 用于控制输出tokens的多样性，TopP值越大输出的tokens类型越丰富，取值范围0~1
                "top_k": 16,  # 选择预测值最大的k个token进行采样，取值范围0-1024，0表示不生效
            },
            "messages": [
                {
                    "role": ChatRole.USER,
                    "content": prompt
                }
            ]
        }
        
        response = self.test_chat(maas, req)
        # response = requests.post(f"http://js-perf.cn:7001/test/{self._llm_type}", {
        #     "ask": prompt
        # })
        # if self._llm_type == "chatglm":
        #     self.log('<--------chatglm------------')
        #     self.log(response)
        #     return response
        # else:
        #     return "不支持该类型的 llm"
        return response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": 10}


In [55]:
'''
Usage:

1. python3 -m pip install --user volcengine
2. VOLC_ACCESSKEY=XXXXX VOLC_SECRETKEY=YYYYY python main.py
'''
import os
from volcengine.maas import MaasService, MaasException, ChatRole


def test_chat(maas, req):
    try:
        resp = maas.chat(req)
        print(resp)
        print(resp.choice.message.content)
    except MaasException as e:
        print(e)

    
def test_stream_chat(maas, req):
    try:
        resps = maas.stream_chat(req)
        for resp in resps:
            print(resp)
            print(resp.choice.message.content)
    except MaasException as e:
        print(e)


if __name__ == '__main__':
    maas = MaasService('maas-api.ml-platform-cn-beijing.volces.com', 'cn-beijing')
    
    maas.set_ak("AKLTNjU0MmRhMjRlNTViNGM1OGIwMzE2ZmIzYzQyNWM1ODk")
    maas.set_sk("T0RjMVpUUTVaRE0xTlRBME5EQXlOMkUzWkdObU56UTFOekkyTlRNd1pqVQ==")
    
    # document: "https://www.volcengine.com/docs/82379/1099475"
    req = {
        "model": {
            "name": "chatglm2-pro",
        },
        "parameters": {
            "max_new_tokens": 1024,  # 输出文本的最大tokens限制
            "temperature": 0.8,  # 用于控制生成文本的随机性和创造性，Temperature值越大随机性越大，取值范围0~1
            "top_p": 0.8,  # 用于控制输出tokens的多样性，TopP值越大输出的tokens类型越丰富，取值范围0~1
            "top_k": 16,  # 选择预测值最大的k个token进行采样，取值范围0-1024，0表示不生效
        },
        "messages": [
            {
                "role": ChatRole.USER,
                "content": "天为什么这么蓝？"
            }, {
                "role": ChatRole.ASSISTANT,
                "content": "因为有你"
            }, {
                "role": ChatRole.USER,
                "content": "花儿为什么这么香？"
            },
        ]
    }
    
    test_chat(maas, req)
    test_stream_chat(maas, req)


{'req_id': '2024042714392469611776F5B8669EF57B', 'choice': {'message': {'content': '因为有蜜蜂'}, 'finish_reason': 'stop'}, 'usage': {'prompt_tokens': 46, 'completion_tokens': 4, 'total_tokens': 50}}
因为有蜜蜂
{'req_id': '20240427143925F352F53A9B5D4D9B7355', 'choice': {'message': {'content': ' '}}}
 
{'req_id': '20240427143925F352F53A9B5D4D9B7355', 'choice': {'message': {'content': '因为有蜜蜂'}, 'finish_reason': 'stop'}, 'usage': {'prompt_tokens': 46, 'completion_tokens': 4, 'total_tokens': 50}}
因为有蜜蜂


In [8]:
import re
from typing import List

from langchain.text_splitter import CharacterTextSplitter


class ChineseTextSplitter(CharacterTextSplitter):
    def __init__(self, pdf: bool = False, **kwargs):
        super().__init__(**kwargs)
        self.pdf = pdf

    def split_text(self, text: str) -> List[str]:
        if self.pdf:
            text = re.sub(r"\n{3,}", "\n", text)
            text = re.sub('\s', ' ', text)
            text = text.replace("\n\n", "")
        sent_sep_pattern = re.compile(
            '([﹒﹔﹖﹗．。！？]["’”」』]{0,2}|(?=["‘“「『]{1,2}|$))') 
        sent_list = []
        for ele in sent_sep_pattern.split(text):
            if sent_sep_pattern.match(ele) and sent_list:
                sent_list[-1] += ele
            elif ele:
                sent_list.append(ele)
        return sent_list

In [3]:
import os

LOCAL_CONTENT = os.path.join("/opt/tiger/rag/docs")
VS_PATH = os.path.join("/opt/tiger/rag/vector_store/FAISS")
CHUNK_SIZE = 800
CHUNK_OVERLAP = 70
VECTOR_SEARCH_TOP_K = 2
os.environ["SERPAPI_API_KEY"] = "118c9b24c853f93c5bc138d14dd4af767e9ce6ac41caa211218af4763f9d86f6"

PROMPT_TEMPLATE = """已知信息：
{context}, 回答该问题:{question}"""

In [77]:
from typing import Any, List, Dict, Mapping, Optional
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.vectorstores import FAISS
# from models.custom_llm import CustomLLM
import datetime
import torch
from tqdm import tqdm
# from models.config import *
from langchain import PromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain

conversation_template = """你是一个正在跟某个人类对话的机器人.

{chat_history}
人类: {human_input}
机器人:"""

def load_txt_file(filepath):
    loader = TextLoader(filepath, encoding="utf8")
    textsplitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE,
                                         chunk_overlap=CHUNK_OVERLAP,
                                         length_function=len)
    docs = loader.load_and_split(text_splitter=textsplitter)
    return docs

def torch_gc():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    elif torch.backends.mps.is_available():
        try:
            from torch.mps import empty_cache
            empty_cache()
        except Exception as e:
            print(e)
            print("如果您使用的是 macOS 建议将 pytorch 版本升级至 2.0.0 或更高版本，以支持及时清理 torch 产生的内存占用。")

def load_file(filepath):
    if filepath.lower().endswith(".md"):
        loader = UnstructuredFileLoader(filepath, mode="elements")
        docs = loader.load()
    elif filepath.lower().endswith(".pdf"):
        loader = UnstructuredFileLoader(filepath)
        textsplitter = ChineseTextSplitter(pdf=True)
        docs = loader.load_and_split(textsplitter)
    else:
        docs = load_txt_file(filepath)
    return docs

def get_related_content(related_docs):
    related_content = []
    for doc in related_docs:
        related_content.append(doc.page_content)
    return "\n".join(related_content)

def get_docs_with_score(docs_with_score):
    docs = []
    for doc, score in docs_with_score:
        doc.metadata["score"] = score
        docs.append(doc)
    return docs

# filepath 可以是目录，也可以是文件
def init_knowledge_vector_store(filepath: str or List[str],
                                vs_path: str or os.PathLike = None,
                                embeddings: object = None):
    loaded_files = []
    failed_files = []
    # 单个文件
    if isinstance(filepath, str):
        if not os.path.exists(filepath):
            print(f"{filepath} 路径不存在")
            return None
        elif os.path.isfile(filepath):
            file = os.path.split(filepath)[-1]
            try:
                docs = load_file(filepath)
                print(f"{file} 已成功加载")
                loaded_files.append(filepath)
            except Exception as e:
                print(e)
                print(f"{file} 未能成功加载")
                return None
        elif os.path.isdir(filepath):
            docs = []
            for file in tqdm(os.listdir(filepath), desc="加载文件"):
                fullfilepath = os.path.join(filepath, file)

                try:
                    docs += load_file(fullfilepath)
                    loaded_files.append(fullfilepath)
                except Exception as e:
                    failed_files.append(file)

            if len(failed_files) > 0:
                print("以下文件未能成功加载：")
                for file in failed_files:
                    print(file,end="\n")
    #  文件列表
    else:
        docs = []
        for file in filepath:
            try:
                docs += load_file(file)
                print(f"{file} 已成功加载")
                loaded_files.append(file)
            except Exception as e:
                print(e)
                print(f"{file} 未能成功加载")

    if len(docs) > 0:
        print("文件加载完毕，正在生成向量库")
        if vs_path and os.path.isdir(vs_path):
            vector_store = FAISS.load_local(vs_path, embeddings, allow_dangerous_deserialization=True)
            vector_store.add_documents(docs)
            torch_gc()
        else:
            if not vs_path:
                vs_path = os.path.join(vs_path,
                                       f"""FAISS_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}""")
            vector_store = FAISS.from_documents(docs, embeddings)
            torch_gc()

        vector_store.save_local(vs_path)
        print("向量生成成功")
        return vs_path, loaded_files
    else:
        print("文件均未成功加载，请检查依赖包或替换为其他文件再次上传。")
        return None, loaded_files


class LocalDocQA:
    filepath: str
    vs_path: str
    load_files: List[str] = []
    top_k: int
    embedding: object
    llm: object
    conversation_with_summary: object
    init: bool = True

    def __init__(self, filepath: str, vs_path: str, embeddings: object,
                       init: bool = True):
        if init:
            vs_path, loaded_files = init_knowledge_vector_store(filepath=LOCAL_CONTENT,
                                                                vs_path=VS_PATH,
                                                                embeddings=embeddings)
        else:
            vs_path = VS_PATH
            loaded_files = []


        self.load_files = loaded_files
        self.vs_path = vs_path
        self.filepath = filepath
        self.embeddings = embeddings
        self.top_k = VECTOR_SEARCH_TOP_K
        self.llm = CustomLLM()
        self.conversation_with_summary = ConversationChain(llm=self.llm,
                                                       memory=ConversationSummaryBufferMemory(llm=self.llm,
                                                                                              max_token_limit=40),
                                                       verbose=True)

    def query_knowledge(self, query: str):
        print(self.vs_path)
        vector_store = FAISS.load_local(self.vs_path, self.embeddings, allow_dangerous_deserialization=True)
        vector_store.chunk_size = CHUNK_SIZE
        related_docs_with_score = vector_store.similarity_search_with_score(query, k = self.top_k)
        related_docs = get_docs_with_score(related_docs_with_score)
        related_content = get_related_content(related_docs)
        return related_content

    def get_knowledge_based_answer(self, query: str):
        related_content = self.query_knowledge(query)
        prompt = PromptTemplate(
            input_variables=["context","question"],
            template=PROMPT_TEMPLATE,
        )
        pmt = prompt.format(context=related_content,
                            question=query)

        # answer=self.conversation_with_summary.predict(input=pmt)
        answer = self.llm(pmt)
        return answer


In [62]:
import requests

RapidAPIKey = "f1047e5fc5msh235fd70b401935fp1e8db7jsn8ee2b109946b"

class DeepSearch:
    def search(self, query=""):
        query = query.strip()

        if query == "":
            return ""

        if RapidAPIKey == "":
            return "请配置你的 RapidAPIKey"

        url = "https://bing-web-search4.p.rapidapi.com/bing-search"
        # print(query)
        payload = {
            "keyword": query,
            "page": 1,
            "lang": "zh-Hans",
            "region": "cn"
        }

        headers = {
            "content-type": "application/json",
            "X-RapidAPI-Key": RapidAPIKey,
            "X-RapidAPI-Host": "bing-web-search4.p.rapidapi.com"
        }

        response = requests.post(url, headers=headers, json=payload)
        # print(response.json())
        data_list = response.json()['search_results']

        if len(data_list) == 0:
            return ""
        else:
            result_arr = []
            result_str = ""
            count_index = 0
            for i in range(6):
                item = data_list[i]
                title = item["title"]
                description = item["caption"]
                item_str = f"{title}: {description}"
                result_arr = result_arr + [item_str]

            result_str = "\n".join(result_arr)
            print(result_str)
            return result_str



In [48]:
ds = DeepSearch()
print(ds.search("microsoft"))

microsoft
Microsoft - Official Home Page: 网页At Microsoft our mission and values are to help people and businesses throughout the world realize their full potential.
Microsoft - 云、计算机、应用和游戏: 网页Visual Studio 2022. 为使用 Windows 系统的 .NET 和 C++ 开发人员提供功能全面的 IDE，助其构建 web、云、桌面设备应用、移动应用、服务和游戏。. 使用最新的 Microsoft AI 解 …
Microsoft account | 立即登录或创建帐户 – Microsoft: 网页Microsoft 帐户连接所有 Microsoft 应用和服务。. 登录以管理帐户。.
Microsoft微软官网Surface_Windows_Office_Microsoft 365 ...: 网页微软商店Microsoft Store,Windows官网,购买Surface平板笔记本二合一电脑官方正品,正版Microsoft 365/Win11系统/Office/Office 2021下载激活等尽在微软官方商城.
Microsoft 365 - Office 应用的订阅 | Microsoft 365: 网页Microsoft 365 订阅包含位于同一个位置的一套熟悉的 Office 应用、智能云服务和世界级安全性。找到适合你的计划。
下载 Windows 10: 网页如果没有安装 Windows 10 的许可证，并且以前尚未升级到此版本，则你可以在此处购买一份：https://www.microsoft.com/zh-cn/windows/get-windows-10. 如果你之前 …
Microsoft - Official Home Page: 网页At Microsoft our mission and values are to help people and businesses throughout the world realize their full potential.
Microsoft - 云、计算机、应用和游戏: 网页Visual St

In [64]:

from langchain.agents import Tool
from langchain.tools import BaseTool
from langchain import PromptTemplate, LLMChain
# from models.custom_search import DeepSearch
from langchain.agents import BaseSingleActionAgent, AgentOutputParser, LLMSingleActionAgent, AgentExecutor
from typing import List, Tuple, Any, Union, Optional, Type
from langchain.schema import AgentAction, AgentFinish
from langchain.prompts import StringPromptTemplate
from langchain.callbacks.manager import CallbackManagerForToolRun
# from models.custom_llm import CustomLLM
import re

agent_template = """
你现在是一个{role}。这里是一些已知信息：
{related_content}
{background_infomation}
{question_guide}：{input}

{answer_format}
"""

class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        # 没有互联网查询信息
        if len(intermediate_steps) == 0:
            background_infomation = "\n"
            role = "傻瓜机器人"
            question_guide = "我现在有一个问题"
            answer_format = "如果你知道答案，请直接给出你的回答！如果你不知道答案，请你只回答\"DeepSearch('搜索词')\"，并将'搜索词'替换为你认为需要搜索的关键词，除此之外不要回答其他任何内容。\n\n下面请回答我上面提出的问题！"

        # 返回了背景信息
        else:
            # 根据 intermediate_steps 中的 AgentAction 拼装 background_infomation
            background_infomation = "\n\n你还有这些已知信息作为参考：\n\n"
            action, observation = intermediate_steps[0]
            background_infomation += f"{observation}\n"
            role = "聪明的 AI 助手"
            question_guide = "请根据这些已知信息回答我的问题"
            answer_format = ""

        kwargs["background_infomation"] = background_infomation
        kwargs["role"] = role
        kwargs["question_guide"] = question_guide
        kwargs["answer_format"] = answer_format
        return self.template.format(**kwargs)

class CustomSearchTool(BaseTool):
    name: str = "DeepSearch"
    description: str = ""

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None):
        return DeepSearch.search(query = query)

    async def _arun(self, query: str):
        raise NotImplementedError("DeepSearch does not support async")

class CustomAgent(BaseSingleActionAgent):
    @property
    def input_keys(self):
        return ["input"]

    def plan(self, intermedate_steps: List[Tuple[AgentAction, str]],
            **kwargs: Any) -> Union[AgentAction, AgentFinish]:
        return AgentAction(tool="DeepSearch", tool_input=kwargs["input"], log="")

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # group1 = 调用函数名字
        # group2 = 传入参数
        match = re.match(r'^[\s\w]*(DeepSearch)\(([^\)]+)\)', llm_output, re.DOTALL)

        # 如果 llm 没有返回 DeepSearch() 则认为直接结束指令
        if not match:
            return AgentFinish(
                return_values={"output": llm_output.strip()},
                log=llm_output,
            )
        # 否则的话都认为需要调用 Tool
        else:
            action = match.group(1).strip()
            action_input = match.group(2).strip()
            return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)


class DeepAgent:
    tool_name: str = "DeepSearch"
    agent_executor: any
    tools: List[Tool]
    llm_chain: any

    def query(self, related_content: str = "", query: str = ""):
        tool_name = self.tool_name
        result = self.agent_executor.run(related_content=related_content, input=query ,tool_name=self.tool_name)
        return result

    def __init__(self, **kwargs):
        llm = CustomLLM()
        tools = [
                    Tool.from_function(
                        func=DeepSearch.search,
                        name="DeepSearch",
                        description=""
                    )
                ]
        self.tools = tools
        tool_names = [tool.name for tool in tools]
        output_parser = CustomOutputParser()
        prompt = CustomPromptTemplate(template=agent_template,
                                      tools=tools,
                                      input_variables=["related_content","tool_name", "input", "intermediate_steps"])

        llm_chain = LLMChain(llm=llm, prompt=prompt)
        self.llm_chain = llm_chain

        agent = LLMSingleActionAgent(
            llm_chain=llm_chain,
            output_parser=output_parser,
            stop=["\nObservation:"],
            allowed_tools=tool_names
        )

        agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
        self.agent_executor = agent_executor


if __name__ == "__main__":
    # from custom_llm import CustomLLM

    llm = CustomLLM()
    tools = [
                Tool.from_function(
                    func=DeepSearch.search,
                    name="DeepSearch",
                    description=""
                )
            ]
    tool_names = [tool.name for tool in tools]
    output_parser = CustomOutputParser()
    prompt = CustomPromptTemplate(template=agent_template,
                                  tools=tools,
                                  input_variables=["related_content","tool_name", "input", "intermediate_steps"])

    llm_chain = LLMChain(llm=llm, prompt=prompt)

    agent = LLMSingleActionAgent(
        llm_chain=llm_chain,
        output_parser=output_parser,
        stop=["\nObservation:"],
        allowed_tools=tool_names
    )

    agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
    # print("hi")
    print(agent_executor.run(related_content="", input="介绍一下微软", tool_name="DeepSearch"))





> Entering new AgentExecutor chain...
{'req_id': '202404271446106F4862E46CD42A724E67', 'choice': {'message': {'content': '微软是一家世界知名的软件公司,主要业务是开发和销售计算机软件、个人电脑和消费电子产品。微软的旗舰产品包括操作系统和办公软件套件,如Windows和Microsoft Office。微软还开发了许多其他知名软件,如Internet Explorer浏览器、Skype网络电话和Xbox游戏机。微软于1975年创立,总部位于美国华盛顿州西雅图市。'}, 'finish_reason': 'stop'}, 'usage': {'prompt_tokens': 98, 'completion_tokens': 85, 'total_tokens': 183}}
微软是一家世界知名的软件公司,主要业务是开发和销售计算机软件、个人电脑和消费电子产品。微软的旗舰产品包括操作系统和办公软件套件,如Windows和Microsoft Office。微软还开发了许多其他知名软件,如Internet Explorer浏览器、Skype网络电话和Xbox游戏机。微软于1975年创立,总部位于美国华盛顿州西雅图市。
微软是一家世界知名的软件公司,主要业务是开发和销售计算机软件、个人电脑和消费电子产品。微软的旗舰产品包括操作系统和办公软件套件,如Windows和Microsoft Office。微软还开发了许多其他知名软件,如Internet Explorer浏览器、Skype网络电话和Xbox游戏机。微软于1975年创立,总部位于美国华盛顿州西雅图市。

> Finished chain.
微软是一家世界知名的软件公司,主要业务是开发和销售计算机软件、个人电脑和消费电子产品。微软的旗舰产品包括操作系统和办公软件套件,如Windows和Microsoft Office。微软还开发了许多其他知名软件,如Internet Explorer浏览器、Skype网络电话和Xbox游戏机。微软于1975年创立,总部位于美国华盛顿州西雅图市。


In [81]:
import torch.cuda
import torch.backends
from typing import Any, List, Dict, Union, Mapping, Optional
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# from models.custom_llm import CustomLLM
# from models.custom_agent import DeepAgent
# from models.util import LocalDocQA
# from models.config import *
EMBEDDING_DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

deep_agent = DeepAgent()

embeddings = HuggingFaceEmbeddings(model_name="GanymedeNil/text2vec-base-chinese",
                                   model_kwargs={'device':EMBEDDING_DEVICE})

qa_doc = LocalDocQA(filepath=LOCAL_CONTENT,
                    vs_path=VS_PATH,
                    embeddings=embeddings,
                    init=True)

def answer(query: str = ""):
    question = query
    related_content = qa_doc.query_knowledge(query=question)
    formed_related_content = "\n" + related_content
    result = deep_agent.query(related_content=formed_related_content, query=question)
    return result

No sentence-transformers model found with name GanymedeNil/text2vec-base-chinese. Creating a new one with MEAN pooling.
加载文件: 100%|██████████| 2/2 [00:00<00:00, 123.88it/s]


文件加载完毕，正在生成向量库
向量生成成功


In [82]:
print(answer("介绍奥巴马"))

/opt/tiger/rag/vector_store/FAISS


> Entering new AgentExecutor chain...
{'req_id': '2024042714574230D4280DBFBF177E4AA8', 'choice': {'message': {'content': '贝拉克·奥巴马（Barack Obama），全名贝拉克·侯赛因·奥巴马二世（Barack Hussein Obama II），生于1961年8月4日，美国政治家、律师，第44任美国总统，于2009年至2017年担任总统。奥巴马是美国历史上第一位非洲裔总统，他的总统任期内，美国经济逐渐从金融危机中复苏，奥巴马政府推行了一系列改革，如奥巴马医保、跨太平洋战略经济伙伴关系协议等。奥巴马卸任后，仍活跃在政治圈，经常发表演讲并会晤其他国家元首。奥巴马信奉基督教，热爱高尔夫球运动。他与夫人米歇尔·奥巴马育有两个女儿，玛丽亚和萨沙。'}, 'finish_reason': 'stop'}, 'usage': {'prompt_tokens': 1078, 'completion_tokens': 152, 'total_tokens': 1230}}
贝拉克·奥巴马（Barack Obama），全名贝拉克·侯赛因·奥巴马二世（Barack Hussein Obama II），生于1961年8月4日，美国政治家、律师，第44任美国总统，于2009年至2017年担任总统。奥巴马是美国历史上第一位非洲裔总统，他的总统任期内，美国经济逐渐从金融危机中复苏，奥巴马政府推行了一系列改革，如奥巴马医保、跨太平洋战略经济伙伴关系协议等。奥巴马卸任后，仍活跃在政治圈，经常发表演讲并会晤其他国家元首。奥巴马信奉基督教，热爱高尔夫球运动。他与夫人米歇尔·奥巴马育有两个女儿，玛丽亚和萨沙。
贝拉克·奥巴马（Barack Obama），全名贝拉克·侯赛因·奥巴马二世（Barack Hussein Obama II），生于1961年8月4日，美国政治家、律师，第44任美国总统，于2009年至2017年担任总统。奥巴马是美国历史上第一位非洲裔总统，他的总统任期内，美国经济逐渐从金融危机中复苏，奥巴马政府推行了一系列改革，如奥巴马医保、跨太平洋战略经济伙伴关系协议等。奥巴马卸任后，仍活跃在政治圈，经常

In [ ]:
from flask import Flask, jsonify, request, make_response
import json

app = Flask(__name__)

@app.route('/ai/langchain/', methods=["POST"])
def handle_langchain_ask():
    if not request.form or not 'ask' in request.form:
        return make_response(jsonify({ "status": 500,
                                       "error":"error form params"}), 500)
    ask = request.form.get('ask')
    content = answer(ask)
    return make_response(jsonify({ 'status': 200,
                                   'content': content}), 200)

if __name__ == "__main__":
    app.run(debug=False, port=8899, host="127.0.0.1")